# The Power of Delimiters

Welcome! We know that a good prompt is built on the "Three Pillars" of Instruction, Context, and Constraints. But how do we organize these pillars within the prompt itself to ensure the model understands where one ends and the next begins?

The answer is one of the simplest yet most effective techniques in prompt engineering: **using delimiters.**

A delimiter is just a set of characters that marks the boundary of a piece of text. By using them, you transform your prompt from a confusing "wall of text" into a structured, well-organized document that is easy for the model to parse. This simple act dramatically increases the reliability of your outputs.

In this notebook, we'll see the power of delimiters in action. We'll start with a poorly structured prompt and show how adding different types of delimiters makes it far more effective.

## Setup: Helper Functions and Context

First, let's set up our reusable helper functions and define the context for our task. The task will be a common one for developers: **summarizing a technical JSON API response into a human-readable bug report.**

In [ ]:
import os
import litellm
import json
from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 200

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        max_tokens=max_tokens,
        **kwargs
    )

    return response.choices[0].message.content

# --- Our Context Data ---

bug_ticket_json = {
  "ticket_id": "BUG-4521",
  "status": "Open",
  "priority": "High",
  "component": "api-v2",
  "title": "User authentication fails with expired token",
  "logs": [
    {"timestamp": "2024-07-26T10:00:15Z", "level": "ERROR", "message": "Token validation failed: expired"},
    {"timestamp": "2024-07-26T10:00:15Z", "level": "INFO", "message": "Attempting re-authentication for user: user_abc"}
  ],
  "reporter": "dev_jane"
}

# Convert the Python dict to a JSON string for use in the prompt
bug_ticket_str = json.dumps(bug_ticket_json, indent=2)

print("Helper functions and context data are ready.")

## The "Before": A Poorly Structured Prompt

Let's start by creating a prompt with no clear structure. We'll blend the instruction, the JSON data, and our constraints into a single paragraph. This is a common mistake that can confuse the model and lead to unreliable results, especially when using cheaper, less powerful models. **This can also confuse us, as we need to be much more careful when inspecting and modifying the prompt!**

The model might get this right sometimes, but it's risky. It has to work harder to figure out what part of the prompt is the instruction versus the data to be operated on. This can lead to errors, especially with more complex inputs.

In [ ]:
unstructured_prompt = [
    {
        "role": "user",
        "content": f"""
Please summarize the following bug ticket data for a report. {bug_ticket_str},
and it should start with the ticket ID and title. Mention the priority and
the affected component, but do not include the raw logs in the final summary.
"""
    }
]

response_unstructured = get_completion(unstructured_prompt)

In [ ]:
print(response_unstructured)

## The "After": Adding Structure with Delimiters

Now, let's refactor the *exact same prompt* using different delimiter styles. Notice how much clearer and more organized the prompts become. This clarity is valuable for you as a developer and for the model as the processor.

### Delimiter Style 1: Markdown Headers

Using Markdown-style headers (`###`) is highly readable and is an excellent way to label the different pillars of your prompt.

In [ ]:
prompt_with_markdown = dedent(f"""
# INSTRUCTION
Summarize the bug ticket data into a single paragraph for a report.

# TICKET DATA
{bug_ticket_str}

# CONSTRAINTS
* The summary must be a single paragraph.
* Start the summary with the ticked ID and the title.
* Mention the priority and the affected component.
* Do not include the raw logs in the final summary.
""")

response_md = get_completion(prompt_with_markdown)

In [ ]:
print(response_md)

### Delimiter Style 2: XML Tags

Using custom XML-style tags is extremely explicit and unambiguous for the model. It clearly marks the beginning and end of each block of information.

In [ ]:
prompt_with_xml = dedent(f"""
<instructions>
Summarize the bug ticket data into a single paragraph for a report.
</instructions>

<ticket_data>
{bug_ticket_str}
</ticket_data>

<constraints>
* The summary must be a single paragraph.
* Start the summary with the ticked ID and the title.
* Mention the priority and the affected component.
* Do not include the raw logs in the final summary.
</constraints>
""")

response_xml = get_completion(prompt_with_xml)

In [ ]:
print(response_xml)

### Delimiter Style 3: Triple Backticks

Triple backticks are a developer's favorite. Using them with a language identifier (like `json`) gives the model a strong hint about the format of the context data, which can further improve parsing accuracy.

In [ ]:
prompt_with_backticks = dedent(f"""
Summarize the bug ticket data into a single paragraph for a report.

```json
{bug_ticket_str}
```

Please follow the following rules:
* The summary must be a single paragraph.
* Start the summary with the ticked ID and the title.
* Mention the priority and the affected component.
* Do not include the raw logs in the final summary.
""")

response_backticks = get_completion(prompt_with_backticks)

In [ ]:
print(response_backticks)